In [1]:
!git clone https://github.com/taide-taiwan/taide-bench-eval

Cloning into 'taide-bench-eval'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 88 (delta 38), reused 71 (delta 28), pack-reused 0
Receiving objects: 100% (88/88), 31.63 KiB | 15.82 MiB/s, done.
Resolving deltas: 100% (38/38), done.


In [2]:
!pip install huggingface_hub

In [9]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-creden

In [10]:
!pip install transformers sentencepiece

In [11]:
import torch

In [12]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
model_path = "taide/TAIDE-LX-7B-Chat"
!nvidia-smi
model = AutoModelForCausalLM.from_pretrained(model_path).cuda()
tokenizer = AutoTokenizer.from_pretrained("taide/TAIDE-LX-7B-Chat", use_fast=False)

Thu May  2 15:39:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   44C    P8              12W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/taide/TAIDE-LX-7B-Chat.
403 Client Error. (Request ID: Root=1-6633b3b7-2af2e800226de8e6630a7c13;3ae76c07-ed3e-4dfb-9640-185d1afc24b9)

Cannot access gated repo for url https://huggingface.co/taide/TAIDE-LX-7B-Chat/resolve/main/config.json.
Access to model taide/TAIDE-LX-7B-Chat is restricted and you are not in the authorized list. Visit https://huggingface.co/taide/TAIDE-LX-7B-Chat to ask for access.

In [7]:
!pip install conda
!conda env create -f environment.yml --solver libmamba
!conda activate taide-bench

ERROR: Could not find a version that satisfies the requirement conda (from versions: 3.0.6, 3.5.0, 3.7.0, 3.17.0, 4.0.0, 4.0.1, 4.0.2, 4.0.3, 4.0.4, 4.0.5, 4.0.7, 4.0.8, 4.0.9, 4.1.2, 4.1.6, 4.2.6, 4.2.7, 4.3.13, 4.3.16)
ERROR: No matching distribution found for conda
/bin/bash: line 1: conda: command not found
/bin/bash: line 1: conda: command not found


In [ ]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [ ]:
while True:
  user_input = input('請輸入你的問題：')

  chat = [
    {"role": "system", "content": "{你是一個來自台灣的AI助理，你的名字是 TAIDE，樂於以台灣人的立場幫助使用者，會用繁體中文回答問題。}"},
    {"role": "user", "content": "{user_input}"},
  ]

  # Convert the chat variable to a string using the str.format() method
  chat_str = str.format('{0}', chat)

  prompt = tokenizer.apply_chat_template(chat_str)
  prompt = tokenizer.decode(prompt, skip_special_tokens=True)

  # Move the input tensor to the GPU
  # input_ids = tokenizer(chat_str, return_tensors='pt').input_ids.to(model.device)
  input_ids = tokenizer(chat_str, return_tensors='pt', padding=True, truncation=True).input_ids.to(model.device)
  attn_mask = tokenizer(chat_str, return_tensors='pt', padding=True, truncation=True).attention_mask.to(model.device)

  # generated_id = model.generate(input_ids, max_new_tokens=1000, streamer=streamer)
  generated_id = model.generate(input_ids, attention_mask=attn_mask, max_new_tokens=1000, pad_token_id=tokenizer.eos_token_id)
  print(tokenizer.decode(generated_id[0], skip_special_tokens=True))

請輸入你的問題：你會中文嗎
[{'role': 'system', 'content': '{你是一個來自台灣的AI助理，你的名字是 TAIDE，樂於以台灣人的立場幫助使用者，會用繁體中文回答問題。}'}, {'role': 'user', 'content': '{user_input}'}] 
請輸入你的問題：0
[{'role': 'system', 'content': '{你是一個來自台灣的AI助理，你的名字是 TAIDE，樂於以台灣人的立場幫助使用者，會用繁體中文回答問題。}'}, {'role': 'user', 'content': '{user_input}'}] 


KeyboardInterrupt: Interrupted by user